## main_pendulum_T_0_01

In [ ]:
import os, sys
print(os.getcwd())
# Add the current directory to sys.path
sys.path.append(os.getcwd() + '/case_study_pendulum')
print(sys.executable)

import numpy as np

from pybasin.Sampler import GridSampler
from pybasin.BasinStabilityEstimator import BasinStabilityEstimator

from case_study_pendulum.setup_pendulum_system import setup_pendulum_system



/Users/adrian/Documents/code/bSTAB/thesis
/Users/adrian/.local/share/virtualenvs/thesis-8NJDacTB/bin/python


In [7]:
props = setup_pendulum_system()
sampler = GridSampler(
    min_limits=(-np.pi + np.arcsin(props["ode_system"].params["T"] / props["ode_system"].params["K"]), -10.0),
    max_limits=(np.pi + np.arcsin(props["ode_system"].params["T"] / props["ode_system"].params["K"]), 10.0)
    )


In [8]:
print(props["ode_system"].params["T"])
print(props["ode_system"].params["K"])
-np.pi + np.arcsin(props["ode_system"].params["T"] / props["ode_system"].params["K"])


0.5
1.0


np.float64(-2.6179938779914944)

In [9]:

props["ode_system"].params["T"] = 0.01
props["N"] = 10000

props["sampler"] = sampler

# [-2.94023473279946,-10]
# -2.940234732799462, -10
grid = sampler.sample(props["N"])
print(grid)
print(len(grid))


Created grid with 10000 points (100 points per dimension)
tensor([[ -2.6180, -10.0000],
        [ -2.5545, -10.0000],
        [ -2.4911, -10.0000],
        ...,
        [  3.5383,  10.0000],
        [  3.6017,  10.0000],
        [  3.6652,  10.0000]])
10000


In [ ]:
bse = BasinStabilityEstimator(
    name="main_pendulum_T_0_01",
    N=props["N"],
    ode_system=props["ode_system"],
    sampler=props["sampler"],
    solver=props["solver"],
    feature_extractor=props["feature_extractor"],
    cluster_classifier=props["cluster_classifier"],
    save_to="main_pendulum_T_0_01"
)

In [11]:
bse.estimate_bs()


Starting Basin Stability Estimation...

1. Generating initial conditions...
Created grid with 10000 points (100 points per dimension)
   Generated 10000 initial conditions

2. Integrating ODE system...
[TorchDiffEqSolver] Cache miss. Integrating...
[TorchDiffEqSolver] Integration result cached to file: /Users/adrian/Documents/code/bSTAB/thesis/case_study_pendulum/cache/db3bbe6049146bf2f4b5440418718f4b.pkl
   Integration complete - trajectory shape: torch.Size([25001, 10000, 2])

3. Creating Solution object...

4. Extracting features...
   Features shape: torch.Size([10000, 2])

5. Performing classification...
   Fitting classifier with template data...
classifier.fit - ode_system.params:  {'alpha': 0.1, 'T': 0.5, 'K': 1.0}
classifier.fit - self.initial_conditions:  tensor([[0.5000, 0.0000],
        [2.7000, 0.0000]])
classifier.fit - self.labels:  ['FP', 'LC']
[TorchDiffEqSolver] Loading integration result from persistent cache.
[[1. 0.]
 [0. 1.]] ['FP', 'LC']
   Classification comple

{'FP': np.float64(1.0)}

In [16]:
solution = bse.solution

In [17]:
print(solution.bifurcation_amplitudes)
print(solution.labels)

tensor([[90.2039, 10.0000],
        [90.1906, 10.0000],
        [90.1770, 10.0000],
        ...,
        [96.3943, 10.0000],
        [96.3980, 10.0000],
        [96.4013, 10.0000]])
['FP' 'FP' 'FP' ... 'FP' 'FP' 'FP']


In [23]:
import pandas as pd

# Create a DataFrame with the specified columns
df = pd.DataFrame({
    'Grid Sample': [(x, y) for x, y in grid.tolist()],
    'Labels': solution.labels,
    'Bifurcation Amplitudes': [(theta, theta_dot) for theta, theta_dot in solution.bifurcation_amplitudes.tolist()]
})

print(df)

df.to_excel("main_pendulum_T_0_01.xlsx")

                       Grid Sample Labels     Bifurcation Amplitudes
0     (-2.6179938316345215, -10.0)     FP  (90.20388793945312, 10.0)
1     (-2.5545272827148438, -10.0)     FP  (90.19064331054688, 10.0)
2      (-2.491060733795166, -10.0)     FP   (90.1769790649414, 10.0)
3     (-2.4275941848754883, -10.0)     FP  (90.16317749023438, 10.0)
4     (-2.3641278743743896, -10.0)     FP  (90.14917755126953, 10.0)
...                            ...    ...                        ...
9995     (3.411325454711914, 10.0)     FP   (96.3858642578125, 10.0)
9996    (3.4747917652130127, 10.0)     FP   (96.3902587890625, 10.0)
9997    (3.5382583141326904, 10.0)     FP    (96.394287109375, 10.0)
9998     (3.601724863052368, 10.0)     FP  (96.39798736572266, 10.0)
9999     (3.665191411972046, 10.0)     FP   (96.4013442993164, 10.0)

[10000 rows x 3 columns]
